In [3]:
# Install necessary libraries
!pip install pdfplumber pandas openai faiss-cpu transformers sentence-transformers streamlit gradio langchain


EXTRACT P&L DATA FROM PDF

In [4]:
import pdfplumber

# Load the PDF file
pdf_path = "Sample Financial Statement.pdf"

# Extract P&L data
try:
    with pdfplumber.open(pdf_path) as pdf:
        # Select the page containing P&L data (e.g., page 2)
        page = pdf.pages[1]  # Adjust the page index as needed
        text = page.extract_text()

    # Clean the text for encoding issues
    cleaned_text = text.encode("ascii", "ignore").decode("utf-8")

    # Save the cleaned text to a file
    with open("extracted_pnl_cleaned.txt", "w", encoding="utf-8") as f:
        f.write(cleaned_text)

    print("P&L data extracted and saved successfully!")

except UnicodeEncodeError as e:
    print(f"Encoding error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")


P&L data extracted and saved successfully!


In [5]:
import pandas as pd

# Load the cleaned text file
file_path = "extracted_pnl_cleaned.txt"

with open(file_path, "r", encoding="utf-8") as f:
    cleaned_text = f.read()

# Split the text into lines
lines = cleaned_text.splitlines()

# Process each line to filter meaningful financial data
data = []
for line in lines:
    # Check if the line contains financial data (adjust filter criteria as needed)
    if "₹" in line or any(char.isdigit() for char in line):  # Properly close the condition
        parts = line.split()  # Split line into parts based on spaces
        data.append(parts)

# Find the maximum number of columns in the data
max_columns = max(len(row) for row in data)

# Create a DataFrame with dynamic column handling
df = pd.DataFrame(data, columns=[f"Column_{i}" for i in range(max_columns)])

# Preview the DataFrame
print("Parsed DataFrame:")
print(df.head())

# Save to CSV for further processing
df.to_csv("raw_pnl_data.csv", index=False)
print("Raw structured data saved to raw_pnl_data.csv.")


Parsed DataFrame:
       Column_0          Column_1 Column_2   Column_3 Column_4 Column_5  \
0     Condensed      Consolidated  Balance     Sheets       as       at   
1     Property,             plant      and  equipment      2.2   12,370   
2  Right-of-use            assets     2.19      6,552    6,882     None   
3       Capital  work-in-progress      293        288     None     None   
4      Goodwill               2.3    7,303      7,248     None     None   

  Column_6 Column_7 Column_8 Column_9 Column_10 Column_11 Column_12 Column_13  
0     Note      No.    March      31,      2024     March       31,      2023  
1   13,346     None     None     None      None      None      None      None  
2     None     None     None     None      None      None      None      None  
3     None     None     None     None      None      None      None      None  
4     None     None     None     None      None      None      None      None  
Raw structured data saved to raw_pnl_data.csv.


CLEAN DATASET

In [6]:
# Load raw data
df = pd.read_csv("raw_pnl_data.csv")

# Drop rows with missing values in important columns (adjust based on your data)
df = df.dropna(how="all", subset=["Column_1", "Column_2"])

# Combine related columns into meaningful categories
df["Category"] = df["Column_0"]  # Assume Column_0 holds the category names
df["2024 (₹ crore)"] = df["Column_2"].fillna("") + df["Column_3"].fillna("")
df["2023 (₹ crore)"] = df["Column_4"].fillna("") + df["Column_5"].fillna("")

# Drop unnecessary columns
df = df[["Category", "2024 (₹ crore)", "2023 (₹ crore)"]]

# Clean numerical columns (remove commas and convert to numbers)
df["2024 (₹ crore)"] = pd.to_numeric(df["2024 (₹ crore)"].str.replace(",", ""), errors="coerce")
df["2023 (₹ crore)"] = pd.to_numeric(df["2023 (₹ crore)"].str.replace(",", ""), errors="coerce")

# Drop rows where both numerical columns are empty
df = df.dropna(subset=["2024 (₹ crore)", "2023 (₹ crore)"], how="all")

# Preview the cleaned DataFrame
print("Cleaned DataFrame:")
print(df)

# Save the cleaned data for further processing
df.to_csv("cleaned_pnl_data.csv", index=False)
print("Cleaned structured data saved to cleaned_pnl_data.csv.")


Cleaned DataFrame:
           Category  2024 (₹ crore)  2023 (₹ crore)
1         Property,             NaN    2.212370e+00
2      Right-of-use    2.196552e+00    6.882000e+03
3           Capital    2.932880e+05             NaN
4          Goodwill    7.303725e+07             NaN
5             Other             NaN    1.749000e+03
6       Investments    1.170813e+09             NaN
7             Loans    3.439000e+03             NaN
8             Other             NaN    3.105280e+07
9          Deferred             NaN    4.541245e+06
10           Income             NaN    3.045645e+07
11            Other             NaN    2.121232e+07
12            Total             NaN    5.493500e+04
13      Investments    1.291569e+08             NaN
14            Trade    2.730193e+00    2.542400e+04
15             Cash             NaN    2.814786e+00
16            Loans    2.482890e+05             NaN
17            Other             NaN    1.208512e+09
18           Income             NaN    6.3976

In [7]:
from sentence_transformers import SentenceTransformer

# Load pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for each row in the DataFrame
df["Embeddings"] = df["Category"].apply(lambda x: model.encode(x))

# Save embeddings for retrieval
import pickle
with open("embeddings.pkl", "wb") as f:
    pickle.dump(df.to_dict(orient="records"), f)


In [8]:
import faiss
import numpy as np

# Load embeddings
with open("embeddings.pkl", "rb") as f:
    records = pickle.load(f)

# Extract embeddings and categories
categories = [record["Category"] for record in records]
embeddings = np.array([record["Embeddings"] for record in records], dtype="float32")

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Query the RAG model
def query_rag_model(question, k=1):
    query_embedding = model.encode(question).astype("float32")
    distances, indices = index.search(np.array([query_embedding]), k)
    results = [categories[i] for i in indices[0]]
    
    # Generate response using GPT
    from transformers import GPT2Tokenizer, GPT2LMHeadModel
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")
    
    input_text = f"Question: {question}\nAnswer based on P&L data: {results[0]}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output = gpt_model.generate(input_ids, max_length=100, num_return_sequences=1)
    
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test the model
print(query_rag_model("What is the total revenue for 2024?"))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Question: What is the total revenue for 2024?
Answer based on P&L data: Income from the sale of the stadium to the NFL is $1.5 billion. The NFL is expected to pay $1.5 billion in taxes on the sale of the stadium.
The NFL is expected to pay $1.5 billion in taxes on the sale of the stadium. The NFL is expected to pay $1.5 billion in taxes on the sale of the stadium to the NFL.


In [9]:
import streamlit as st

# Streamlit app
st.title("P&L QA Bot")
uploaded_file = st.file_uploader("Upload a financial statement PDF", type=["pdf"])
question = st.text_input("Ask a question about the P&L data:")

if uploaded_file and question:
    with pdfplumber.open(uploaded_file) as pdf:
        text = pdf.pages[1].extract_text()
    response = query_rag_model(question)
    st.write(f"Answer: {response}")


2025-01-17 12:01:30.283 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-17 12:01:31.189 
  command:

    streamlit run C:\Users\Hp\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-01-17 12:01:31.189 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-17 12:01:31.189 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-17 12:01:31.189 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-17 12:01:31.189 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-17 12:01:31.189 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-17 12:01:31.196 Thread 'MainThread': missi

In [10]:
import gradio as gr

# Gradio app
def process_query(file, query):
    with pdfplumber.open(file) as pdf:
        text = pdf.pages[1].extract_text()
    return query_rag_model(query)

gr.Interface(
    fn=process_query,
    inputs=["file", "text"],
    outputs="text",
    title="P&L QA Bot",
).launch()


Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
